# LLM as a Judge

Based on llm as a judge paper https://arxiv.org/abs/2306.05685, We have implemented this function.

In this notebook we will go over the function's docs and outputs and see an end-to-end example of running it.

1. [Single grading metrics](#chapter1)
2. [Pairwise grading metrics](#chapter2)
3. [Pairwise_with_reference_grading_metrics](#chapter3)

## 0. define the metrics

In [9]:
prompt_config = {
        "name": "accuracy",
        "definition": "The accuracy of the provided answer.",
        "rubric": """Accuracy: This rubric assesses the accuracy of the provided answer. The details for different scores are as follows:
            - Score 1: The answer is completely incorrect or irrelevant to the question. It demonstrates a fundamental
              misunderstanding of the topic or question.
            - Score 2: The answer contains significant inaccuracies, though it shows some understanding of the topic. Key
              elements of the question are addressed incorrectly.
            - Score 3: The answer is partially correct but has noticeable inaccuracies or omissions. It addresses the
              question but lacks depth or precision.
            - Score 4: The answer is mostly correct, with only minor inaccuracies or omissions. It provides a generally
              accurate response to the question.
            - Score 5: The answer is completely correct and thorough. It demonstrates a deep and accurate understanding of
              the topic, addressing all elements of the question effectively.""",
        "examples": """
            Question: What is the capital of France?
            Score 1: Completely Incorrect
            Answer: "The capital of France is Berlin."
            Explanation: This answer is entirely incorrect and irrelevant, as Berlin is the capital of Germany, not France.
            Score 2: Significantly Inaccurate
            Answer: "The capital of France is Lyon."
            Explanation: This answer demonstrates some understanding that the question is about a city in France, but it incorrectly identifies Lyon as the capital instead of Paris.
            Score 3: Partially Correct
            Answer: "I think the capital of France is either Paris or Marseille."
            Explanation: This answer shows partial knowledge but includes a significant inaccuracy by suggesting Marseille might be the capital. Paris is correct, but the inclusion of Marseille indicates a lack of certainty or complete understanding.
            Score 4: Mostly Correct
            Answer: "The capital of France is Paris, the largest city in the country."
            Explanation: This answer is mostly correct and identifies Paris as the capital. The addition of "the largest city in the country" is accurate but not directly relevant to the capital status, introducing a slight deviation from the question's focus.
            Score 5: Completely Correct and Thorough
            Answer: "The capital of France is Paris, which is not only the country's largest city but also its cultural and political center, hosting major institutions like the President's residence, the Elysée Palace."
            Explanation: This answer is completely correct, providing a thorough explanation that adds relevant context about Paris's role as the cultural and political center of France, directly addressing the question with depth and precision.
                     """,
    }

<a id="chapter1"></a>
## 1. Single grading metrics


Single grading metrics will use a self-defined metrics and return a scroe based on the examples and rubric of the metrics

In [10]:
# config of the hugggingface model as judge
JUDGE_MODEL = "TheBloke/Mistral-7B-OpenOrca-GPTQ"
JUDGE_CONFIG = {
    "device_map": "auto",
    "revision": "gptq-8bit-128g-actorder_True",
    "trust_remote_code": False,
}
JUDGE_INFER_CONFIG = {
    "max_length": 1500,
}
TOKENIZER_JUDGE_CONFIG = {"use_fast": True}

In [13]:
import mlrun
llm_judge_fn = mlrun.import_function('function.yaml')

In [14]:
demo1_run = llm_judge_fn.run(
    handler="llm_judge",
    params={
        "input_path": "data/qa.csv",
        "metric_type": "LLMJudgeSingleGrading",
        "name": "accuracy_metrics",
        "model_judge": JUDGE_MODEL,
        "model_judge_config": JUDGE_CONFIG,
        "model_judge_infer_config": JUDGE_INFER_CONFIG,
        "prompt_template" : SINGLE_GRADE_PROMPT,
        "prompt_config" :prompt_config,
        "tokenizer_judge_config": TOKENIZER_JUDGE_CONFIG,
    },
    returns=[
        "result: dataset",
    ],
    local=True,
    artifact_path="./"
)

> 2024-02-15 21:31:51,897 [warning] artifact/output path is not defined or is local and relative, artifacts will not be visible in the UI: {'output_path': './'}
> 2024-02-15 21:31:51,898 [info] Storing function: {'name': 'llm-judge-llm-judge', 'uid': '5199ce0a27d34567a0be952aa0d4c815', 'db': None}
> 2024-02-15 21:31:52,794 [info] Preparing the judge model TheBloke/Mistral-7B-OpenOrca-GPTQ


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


> 2024-02-15 21:32:31,301 [info] Computing the metrics over all data
> 2024-02-15 21:32:31,302 [info] Computing the metrics over one data point with What is the capital of China? and The capital of China is Kongfu
> 2024-02-15 21:32:31,303 [info] Filling the prompt template with the prompt config


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.


> 2024-02-15 21:33:09,392 [info] Extracting the score and explanation from 
Task:
Please act as an impartial judge and evaluate the quality of the response provided by an
AI assistant to the user question displayed below. You will be given the definition of accuracy, grading rubric, context information.
Your task is to determine a numerical score of accuracy for the response. You must use the grading rubric to determine your score. You must also give a explanation about how did you determine the score step-by-step. Please use chain of thinking.
Examples could be included beblow for your reference. Make sure you understand the grading rubric and use the examples before completing the task.
[Examples]:

            Question: What is the capital of France?
            Score 1: Completely Incorrect
            Answer: "The capital of France is Berlin."
            Explanation: This answer is entirely incorrect and irrelevant, as Berlin is the capital of Germany, not France.
            Sco

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


> 2024-02-15 21:34:15,739 [info] Extracting the score and explanation from 
Task:
Please act as an impartial judge and evaluate the quality of the response provided by an
AI assistant to the user question displayed below. You will be given the definition of accuracy, grading rubric, context information.
Your task is to determine a numerical score of accuracy for the response. You must use the grading rubric to determine your score. You must also give a explanation about how did you determine the score step-by-step. Please use chain of thinking.
Examples could be included beblow for your reference. Make sure you understand the grading rubric and use the examples before completing the task.
[Examples]:

            Question: What is the capital of France?
            Score 1: Completely Incorrect
            Answer: "The capital of France is Berlin."
            Explanation: This answer is entirely incorrect and irrelevant, as Berlin is the capital of Germany, not France.
            Sco

> 2024-02-15 21:34:15,974 [info] Run execution finished: {'status': 'completed', 'name': 'llm-judge-llm-judge'}
